In [12]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# GPU 사용 여부 확인
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [14]:
# 현재 작업 디렉토리를 새로운 경로로 변경
import os

path = "/content/drive/MyDrive/Colab Notebooks/fox_star"

os.chdir(path)

HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/Colab Notebooks/fox_star


In [15]:
# YOLO 다운로드를 위해 Ultralytics library 내려 받기
!pip install ultralytics

from IPython import display
display.clear_output() # 라이브러리 버전과 CPU, GPU 외 출력값 제거 위해 사용 

import ultralytics
ultralytics.checks() # ultralytics load 이후 버전 확인

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.0+cpu CPU (Intel Xeon 2.00GHz)
Setup complete ✅ (96 CPUs, 334.6 GB RAM, 15.7/225.3 GB disk)


In [16]:
# 불러오기
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
# # 작업을 위한 폴더 생성 
# # !mkdir {HOME}/datasets
# # %cd {HOME}/datasets

# !mkdir -p '/content/drive/MyDrive/Colab Notebooks/fox_star/datasets'

# # 생성된 폴더로 작업 디렉토리를 변경
# %cd '/content/drive/MyDrive/Colab Notebooks/fox_star/datasets'

/content/drive/MyDrive/Colab Notebooks/fox_star/datasets


In [ ]:
# Roboflow에서 데이터 파일 내려받기 
# 아래에서 학습한 파일과 다른 파일  -> Gray scale를 연기로 착각하는 경우 발생 및 학습 데이터를 늘리기 위해서 dataset 새로 구축 및 정제
!pip install roboflow --quiet

from roboflow import Roboflow

rf = Roboflow(api_key="") # API 키값 제거 함

# Roboflow에서 Project_select
project = rf.workspace("ltttnt").project("fire-vqbia")

# Project_version_select
version = project.version(1) # Project version 중 가장 최신이였던 1로 지정

# Project_model_version_select
dataset = version.download("yolov8") # YOLOv8n의 학습을 위해 YOLOv8용 데이터 파일 다운

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.41, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to FIRE-1 in yolov8:: 100%|██████████| 11432/11432 [02:08<00:00, 88.88it/s]


커스텀 데이터 YAML_FILE 생성

In [ ]:
import yaml # 학습을 위한 커스텀 데이터 셋 생성을 위해 library load 

data = { "train" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/FIRE-1/train/images/", # train image 경로 걸정
        "valid" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/FIRE-1/valid/images/", # valid image 경로 걸정
         "test" : "/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/FIRE-1/test/images/", # test image 경로 걸정
         "names" : ["fire", "smoke"], # 학습 데이터의 Class 이름 설정 앞에서 부터 / Default 순서에 따라 0, 1  
         "nc" : 2 } # 학습시킬 Class의 갯수 



In [17]:
# %cd {HOME} # 모델 내려받을 디렉토리 설정

## 모델 내려 받기
# 객체 탐지를 위해 task = detect
# model = model 버전 설정 -> "model version(yolov1~9)" + "model size(n ,s ,m, l ,x)"
# model size n -> x로 갈 수록 연산이 늘어나 정확도가 올라가지만 성능은 떨어진다. 

from ultralytics import YOLO

model = YOLO('yolov8n.pt')   

In [18]:
# MS.COCO Dataset에 정의되어 있는 Class 개수와 종류 확인
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

In [19]:
# data.yaml 파일 확인 
# train, test, val 경로 및 class name, class num 확인 및 project version 사용
!cat '/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/data.yaml'

train: ../sum_fire/train/images
test: ../sum_fire/test/images
val: ../sum_fire/valid/images



nc: 2
names: ['fire', 'smoke']

roboflow:
  workspace: jabba
  project: fir3
  version: 3
  license: CC BY 4.0
  url: https://universe.roboflow.com/jabba/fir3/dataset/3

In [ ]:
# 기본 문자 인코딩 오류로 인해 local로 불러와 학습을 위해 UTF로 변환
import locale
def getpreferrdencoding(do_setlocale = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferrdencoding

In [ ]:
# 인코딩 여부 확인
print(locale.getpreferredencoding())

In [22]:
# 모델 학습
# Detectinn 할 때 UTF로 되어 있으면 py파일 자체를 못불러옴 -> 그래서 로컬로 불러와서 UTF로 변환하고 출력
# roboflow에서 내려 받은 데이터에서 누락된 부분이 존재하여 현재 실행 화면에서는 오류 발생

model.train(data="/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/data.yaml", # data 파일 경로 지정
            task="detect", # 학습 시킬 taks 선정 
            epochs=3, # epoch 횟수, foward-propagation 이후 back-propagation 해야지 epoch 1번
            patience=1, # early stop을 위해 설정 -> epoch에 따라 설정
            imgsz=416, # 입력 이미지 크기 조절 / 초기의 경우 기본 모델 가져와서 사용해서 모델 구조에 맞춰 416사용
            seed=42, # train, val, test 성능 검증을 위해 set seed 고정
            project="/content/drive/MyDrive/Colab Notebooks/fox_star", # project 생성될 경로 지정
            name="star_train_3", # 생성될 project 이름
            plots=True) # epoch 별 학습에 관한 plot 여부 선정

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/data.yaml, epochs=3, time=None, patience=30, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Colab Notebooks/fox_star, name=star_train_32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/labels... 26417 images, 223 backgrounds, 0 corrupt: 100%|██████████| 26427/26427 [30:38<00:00, 14.37it/s]

train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/0160_jpg.rf.627db855aa971b72bd3be0f9b9abd8b1.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/0160_jpg.rf.8df2b6b375e62af022337870c7b7463d.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/0160_jpg.rf.e435113c4e6ea89e06f9d2d617366a50.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/Img_1996_jpg.rf.0889f48469eddf1c30592da2fad05f6c.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/Img_1996_jpg.rf.45cef38940079c4125664411f85cb9c4.jpg: 2 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/images/Img_1996_jpg.rf.b7adaa18c

train: New cache created: /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 51397, len(boxes) = 59129. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning /content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/valid/labels... 2479 images, 5 backgrounds, 0 corrupt: 100%|██████████| 2479/2479 [02:11<00:00, 18.84it/s]

...

                   all       2479       5496      0.633      0.666      0.687      0.479



3 epochs completed in 1.011 hours.
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_32/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_32/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_32/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.0+cpu CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 78/78 [00:52<00:00,  1.49it/s]


                   all       2479       5496      0.634      0.665      0.687      0.479
                  fire       2479       3242      0.723      0.624       0.72      0.477
                 smoke       2479       2254      0.545      0.706      0.654      0.481
Speed: 0.6ms preprocess, 8.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/fox_star/star_train_32


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7da1509c7430>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

# 성능 확인하기

- 팀원과 결과 공유를 위해 후에 아래 코드 대신 google drive 내의 결과 png 사용

In [ ]:
!ls {HOME}/runs/detect/train/  # 학습 결과 확인을 위해 생성된 project 폴더 값 불러오기 

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600) # model의 성능지표(confusion matrix) 확인

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600) # model의 결과 확인

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600) # model의 예측값 확인

## Validate Custom Model

In [ ]:
# 검증을 위해 경로 확인
HOME

'/content/drive/MyDrive/Colab Notebooks/fox_star'

In [23]:
# model 학습 프로젝트에 따라 경로 수정해줄 것
%cd {HOME}

# model 검증 수행
# mode : train, val 중 검증을 위해 val 사용
# model : train 이후 생성된 project 폴더의 결과 값 사용
# data : 검증 data set의 경로를 위해 train 시 사용한 yaml 값 사용
!yolo task=detect mode=val model="/content/drive/MyDrive/Colab Notebooks/fox_star/star_train_3/weights/best.pt" data="/content/drive/MyDrive/Colab Notebooks/fox_star/datasets/sum_fire/data.yaml"

/content
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 555, in entrypoint
    model = YOLO(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/models/yolo/model.py", line 23, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 151, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 240, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 806, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # load ckpt
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 732, in torch_safe_load
    ckpt = torc

## Inference with Custom Model

In [ ]:
# 모델 결과 학인
%cd {HOME}

!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True